## **1. Setup and WordNet Resources**
#### We need wordnet for the definitions and punkt_tab for the underlying tokenization.

In [3]:
import nltk
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize

# Ensure resources are available
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('punkt_tab')

## **2. Understanding Synsets (Word Senses)**
#### Before disambiguating, let's see why it's hard. A single word like "bank" has many "Synsets" (sets of synonyms).

In [5]:
from nltk.corpus import wordnet as wn

word = "bank"
syns = wn.synsets(word)

print(f"Senses for '{word}':")
for i, syn in enumerate(syns[:5]): # Showing first 5
    print(f"{i+1}. {syn.name()}: {syn.definition()}")

Senses for 'bank':
1. bank.n.01: sloping land (especially the slope beside a body of water)
2. depository_financial_institution.n.01: a financial institution that accepts deposits and channels the money into lending activities
3. bank.n.03: a long ridge or pile
4. bank.n.04: an arrangement of similar objects in a row or in tiers
5. bank.n.05: a supply or stock held in reserve for future use (especially in emergencies)


## **3. Implementing the Lesk Algorithm**
#### The lesk function takes a sentence and the specific word you want to disambiguate. It returns the synset that has the highest overlap between the sentence and the dictionary definition.

In [6]:
def disambiguate_word(sentence, target_word):
    tokens = word_tokenize(sentence)
    best_sense = lesk(tokens, target_word)
    
    if best_sense:
        print(f"Context: {sentence}")
        print(f"Target Word: '{target_word}'")
        print(f"Disambiguated Meaning: {best_sense.definition()}")
        print("-" * 30)
    else:
        print(f"Could not find a sense for '{target_word}'")

# Case 1: Financial context
sent1 = "I need to go to the bank to deposit my paycheck."
disambiguate_word(sent1, 'bank')

# Case 2: Geographic context
sent2 = "The fisherman sat on the river bank watching the water flow."
disambiguate_word(sent2, 'bank')

Context: I need to go to the bank to deposit my paycheck.
Target Word: 'bank'
Disambiguated Meaning: cover with ashes so to control the rate of burning
------------------------------
Context: The fisherman sat on the river bank watching the water flow.
Target Word: 'bank'
Disambiguated Meaning: a container (usually with a slot in the top) for keeping money at home
------------------------------


## **4. Analysis: Why does this matter?**
#### Without WSD, a computer might treat "bank" (money) and "bank" (river) as the same concept.

### **Comparison Table: WSD Impact**

| Feature 	| Without WSD 	| With WSD (Lesk) |
|---|---|---|
|Search Engines	|Returns river maps when you want a loan.	|Filters results based on "finance" intent. |
|Translation	|Translates both to the same word (incorrectly).	|Chooses the correct target word in French/Spanish.|
|Sentiment Analysis	|Confuses "crane" (bird) with "crane" (machinery).	|Correctly identifies the subject of the text.|


## **5. Advanced Application: Batch Disambiguation**
#### You can loop through a sentence to try and find the sense for every noun to build a "semantic profile" of a sentence.

In [7]:
def semantic_profile(sentence):
    tokens = word_tokenize(sentence)
    profile = []
    
    # We only disambiguate words that exist in WordNet
    for word in tokens:
        sense = lesk(tokens, word)
        if sense:
            profile.append((word, sense.name(), sense.definition()))
            
    import pandas as pd
    return pd.DataFrame(profile, columns=['Word', 'Synset', 'Definition'])

text = "The plant needs water to grow in the green forest."
print(semantic_profile(text))

     Word           Synset                                         Definition
0   plant       plant.n.03  an actor situated in the audience whose acting...
1   needs  motivation.n.01  the psychological feature that arouses an orga...
2   water       water.n.03  once thought to be one of four elements compos...
3    grow        grow.v.04                           cause to grow or develop
4      in          in.r.01                         to or toward the inside of
5   green       green.s.01  of the color between blue and yellow in the co...
6  forest      forest.n.01  the trees and other plants in a large densely ...
